# Real life data

## Preprocessing

In [63]:
%load_ext autoreload 
%autoreload 2

import re
import sys
import hashlib

import pandas as pd
import numpy as np
from pathlib import Path

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")
from src import aposteriori

import tasks.run_helper
import tasks.preprocessing

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
df = pd.read_pickle("../data/attitudes_embedded.csv")
df

,tweet,toyou,toany,annotatorMinority,bert,annotatorPolitics,traditionalism,annotatorRace,annotatorAge,annotatorGender,freeSpeech,harmHateSpeech,intent,lingPurism,racism,racist
0,""" you ugly bitch!!!""- that seems to be my real...","[2, 1, 1, 2]","[3, 3, 1, 2]","[{}, pansexual, disabled, trans, {}, {}]","[0.0047034588642418385, 0.11112377792596817, 0...","[-1.0, -1.0, 1.0, 0.5]","[3, 1, 5, 4]","[black, black, white, white]","[25.0, 35.0, 50.0, 25.0]","[man, man, woman, woman]","[1, 1, 1, 2]","[4, 5, 5, 2]","[1, 1, 1, 2]","[1, 1, 5, 5]","[1, 1, 1, 4]","[1, 1, 1, 1]"
1,"""""""I am happy to report that drowning does kil...","[4, 2, 1, 2, 1, 1]","[4, 2, 1, 2, 2, 1]","[{}, {}, {}, {}, {}, {}]","[0.35706740617752075, 0.032704465091228485, 0....","[-0.5, -0.5, 0.5, 0.5, -1.0, -1.0]","[4, 3, 4, 3, 1, 4]","[black, black, white, white, white, white]","[40.0, 30.0, 40.0, 30.0, 25.0, 25.0]","[woman, woman, woman, man, man, man]","[1, 1, 3, 4, 1, 1]","[4, 4, 3, 1, 5, 4]","[4, 2, 1, 3, 1, 1]","[2, 5, 5, 5, 5, 5]","[1, 1, 5, 5, 1, 1]","[1, 1, 1, 1, 1, 1]"
2,"""""""Let them praise the name of the LORD: for h...","[1, 1, 1, 1, 1, 2]","[2, 1, 1, 1, 1, 3]","[{}, black; female, nan, {}, {}, {}]","[-0.022276882082223892, 0.17076537013053894, 0...","[-0.5, -1.0, 1.0, 0.0, -1.0, -0.5]","[3, 4, 3, 5, 1, 3]","[black, black, white, white, white, white]","[30.0, 40.0, 35.0, 40.0, 25.0, 45.0]","[woman, woman, man, woman, man, woman]","[1, 1, 5, 1, 1, 1]","[4, 5, 1, 5, 5, 4]","[1, 1, 1, 1, 1, 1]","[5, 4, 5, 5, 5, 4]","[1, 1, 5, 1, 1, 1]","[1, 1, 1, 1, 1, 1]"
3,"""""""That was decades ago, Cecil! Anyway, we wan...","[1, 2, 1, 1, 1, 1]","[1, 4, 1, 1, 1, 1]","[{}, black; female, nan, {}, {}, nan]","[0.07474598288536072, 0.24487990140914917, 0.2...","[0.0, -1.0, 1.0, 0.5, -0.5, -1.0]","[4, 4, 3, 4, 5, 5]","[black, black, white, white, white, white]","[35.0, 40.0, 35.0, 40.0, 35.0, 45.0]","[man, woman, man, woman, woman, woman]","[1, 1, 5, 3, 1, 1]","[4, 5, 1, 3, 5, 5]","[1, 1, 1, 1, 1, 1]","[4, 4, 5, 5, 4, 3]","[1, 1, 5, 5, 1, 1]","[1, 1, 1, 1, 1, 1]"
4,"""""""This Jestxh idiot keeps riding me in dms, t...","[1, 3, 3, 3, 4]","[1, 2, 4, 4, 4]","[{}, NONE, No i do not, none, {}]","[0.050774428993463516, 0.08187347650527954, 0....","[-1.0, 1.0, 0.5, 0.5, -1.0]","[5, 3, 5, 5, 5]","[black, black, white, white, white]","[45.0, 45.0, 45.0, 60.0, 40.0]","[man, man, woman, woman, woman]","[1, 5, 1, 1, 1]","[5, 4, 3, 5, 2]","[1, 3, 1, 3, 5]","[5, 3, 4, 5, 5]","[1, 3, 5, 1, 1]","[1, 4, 2, 2, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,women are not dumb bitches,"[1, 5, 2, 3]","[4, 5, 2, 3]","[pansexual, disabled, trans, {}, none, {}]","[0.22457970678806305, 0.3840716481208801, -0.4...","[-1.0, 1.0, 0.5, 0.5]","[1, 5, 5, 5]","[black, black, white, white]","[35.0, 30.0, 45.0, 65.0]","[man, woman, man, woman]","[1, 5, 1, 1]","[5, 5, 5, 5]","[1, 5, 1, 3]","[1, 5, 5, 5]","[1, 5, 1, 1]","[1, 1, 1, 1]"
622,worst part about getting highlights is sitting...,"[1, 3, 1, 1, 1, 1]","[1, 5, 1, 1, 1, 1]","[{}, NONE, {}, {}, {}, none]","[0.1781517118215561, 0.02407737821340561, 0.13...","[-1.0, 1.0, 0.5, 0.5, -0.5, -0.5]","[5, 3, 5, 2, 4, 3]","[black, black, white, white, white, white]","[45.0, 45.0, 35.0, 35.0, 25.0, 50.0]","[man, man, woman, man, man, man]","[1, 5, 1, 2, 1, 1]","[5, 4, 4, 2, 3, 5]","[1, 3, 1, 1, 1, 1]","[5, 3, 5, 2, 5, 4]","[1, 5, 2, 3, 2, 1]","[1, 4, 1, 1, 1, 1]"
623,yids are not as honest as the rest of us,"[5, 5, 4, 5, 4, 1]","[5, 5, 4, 5, 4, 1]","[nan, {}, {}, none, {}, bisexual]","[0.2983386516571045, 0.31099164485931396, -0.4...","[-0.5, -0.5, 0.5, 0.5, -0.5, -0.5]","[3, 5, 2, 5, 4, 3]","[black, black, white, white, white, white]","[30.0, 30.0, 40.0, 60.0, 35.0, 40.0]","[man, man, man, woman, woman, woman]","[1, 1, 5, 1, 2, 1]","[3, 3, 2, 5, 3, 5]","[5, 5, 4, 5, 5, 1]","[5, 3, 5, 5, 4, 5]","[1, 1, 4, 1, 2, 1]","[5, 3, 3, 5, 5, 1]"
624,"you've got to be fucking joking, my room has l...","[3, 3, 1, 1, 2, 1]","[2, 4, 1, 2, 3, 2]","[

In [65]:
df = df.loc[
    :,
    [
        "tweet",
        "racism",
        "annotatorAge",
        "annotatorRace",
        "annotatorGender",
    ],
]
df

,tweet,racism,annotatorAge,annotatorRace,annotatorGender
0,""" you ugly bitch!!!""- that seems to be my real...","[1, 1, 1, 4]","[25.0, 35.0, 50.0, 25.0]","[black, black, white, white]","[man, man, woman, woman]"
1,"""""""I am happy to report that drowning does kil...","[1, 1, 5, 5, 1, 1]","[40.0, 30.0, 40.0, 30.0, 25.0, 25.0]","[black, black, white, white, white, white]","[woman, woman, woman, man, man, man]"
2,"""""""Let them praise the name of the LORD: for h...","[1, 1, 5, 1, 1, 1]","[30.0, 40.0, 35.0, 40.0, 25.0, 45.0]","[black, black, white, white, white, white]","[woman, woman, man, woman, man, woman]"
3,"""""""That was decades ago, Cecil! Anyway, we wan...","[1, 1, 5, 5, 1, 1]","[35.0, 40.0, 35.0, 40.0, 35.0, 45.0]","[black, black, white, white, white, white]","[man, woman, man, woman, woman, woman]"
4,"""""""This Jestxh idiot keeps riding me in dms, t...","[1, 3, 5, 1, 1]","[45.0, 45.0, 45.0, 60.0, 40.0]","[black, black, white, white, white]","[man, man, woman, woman, woman]"
...,...,...,...,...,...
621,women are not dumb bitches,"[1, 5, 1, 1]","[35.0, 30.0, 45.0, 65.0]","[black, black, white, white]","[man, woman, man, woman]"
622,worst part about getting highlights is sitting...,"[1, 5, 2, 3, 2, 1]","[45.0, 45.0, 35.0, 35.0, 25.0, 50.0]","[black, black, white, white, white, white]","[man, man, woman, man, man, man]"
623,yids are not as honest as the rest of us,"[1, 1, 4, 1, 2, 1]","[30.0, 30.0, 40.0, 60.0, 35.0, 40.0]","[black, black, white, white, white, white]","[man, man, man, woman, woman, woman]"
624,"you've got to be fucking joking, my room has l...","[1, 4, 1, 3, 3, 1]","[40.0, 25.0, 40.0, 35.0, 45.0, 35.0]","[black, black, white, white, white, white]","[woman, woman, woman, woman, woman, woman]"


Clear out NaNs from dataset

In [66]:
nan_age = df.annotatorAge.apply(lambda x: np.any(np.isnan(x)))
nan_race = df.annotatorRace.apply(lambda x: "na" in x)
nan_gender = df.annotatorGender.apply(lambda x: "na" in x)

nan_age.sum(), nan_race.sum(), nan_gender.sum()

(41, 41, 39)

In [67]:
df.annotatorRace = df.annotatorRace.apply(lambda x: None if ("na" in x) else x)

df.annotatorGender = df.annotatorGender.apply(
    lambda x: None if ("na" in x) else x
)

df.annotatorAge = df.annotatorAge.apply(
    lambda x: None if np.any([np.isnan(age) for age in x]) else x
).apply(
    lambda x: [int(age) for age in x] if x is not None else None
).apply(lambda ls: pd.cut(ls, bins=[0,20,40,60]) if ls is not None else None)

df["random"] = df.racism.apply(
    lambda x: [np.random.randint(1, 3) for _ in range(len(x))]
)
df= df.dropna()
df

,tweet,racism,annotatorAge,annotatorRace,annotatorGender,random
0,""" you ugly bitch!!!""- that seems to be my real...","[1, 1, 1, 4]","[(20, 40], (20, 40], (40, 60], (20, 40]] Categ...","[black, black, white, white]","[man, man, woman, woman]","[1, 1, 1, 1]"
1,"""""""I am happy to report that drowning does kil...","[1, 1, 5, 5, 1, 1]","[(20, 40], (20, 40], (20, 40], (20, 40], (20, ...","[black, black, white, white, white, white]","[woman, woman, woman, man, man, man]","[2, 2, 1, 1, 1, 1]"
2,"""""""Let them praise the name of the LORD: for h...","[1, 1, 5, 1, 1, 1]","[(20, 40], (20, 40], (20, 40], (20, 40], (20, ...","[black, black, white, white, white, white]","[woman, woman, man, woman, man, woman]","[2, 2, 1, 1, 1, 2]"
3,"""""""That was decades ago, Cecil! Anyway, we wan...","[1, 1, 5, 5, 1, 1]","[(20, 40], (20, 40], (20, 40], (20, 40], (20, ...","[black, black, white, white, white, white]","[man, woman, man, woman, woman, woman]","[2, 1, 1, 2, 2, 2]"
4,"""""""This Jestxh idiot keeps riding me in dms, t...","[1, 3, 5, 1, 1]","[(40, 60], (40, 60], (40, 60], (40, 60], (20, ...","[black, black, white, white, white]","[man, man, woman, woman, woman]","[2, 2, 1, 1, 1]"
...,...,...,...,...,...,...
621,women are not dumb bitches,"[1, 5, 1, 1]","[(20.0, 40.0], (20.0, 40.0], (40.0, 60.0], NaN...","[black, black, white, white]","[man, woman, man, woman]","[1, 2, 2, 2]"
622,worst part about getting highlights is sitting...,"[1, 5, 2, 3, 2, 1]","[(40, 60], (40, 60], (20, 40], (20, 40], (20, ...","[black, black, white, white, white, white]","[man, man, woman, man, man, man]","[1, 1, 1, 2, 2, 1]"
623,yids are not as honest as the rest of us,"[1, 1, 4, 1, 2, 1]","[(20, 40], (20, 40], (20, 40], (40, 60], (20, ...","[black, black, white, white, white, white]","[man, man, man, woman, woman, woman]","[2, 2, 2, 2, 1, 1]"
624,"you've got to be fucking joking, my room has l...","[1, 4, 1, 3, 3, 1]","[(20, 40], (20, 40], (20, 40], (20, 40], (40, ...","[black, black, white, white, white, white]","[woman, woman, woman, woman, woman, woman]","[2, 2, 1, 1, 2, 1]"


In [71]:
tasks.preprocessing.find_inconsistent_rows(
    df, ["annotatorAge", "annotatorRace", "racism", "annotatorGender"]
).sum()

0

## Results

We hypothesize that all of the comments belong in the same discussion.

In [69]:
df2 = df.copy()
df2["fake_index"] = 1

In [70]:
tasks.run_helper.results(
    df2,
    discussion_id_col="fake_index",
    sdb_column="annotatorAge",
    value_col="racism",
    comment_key_col="tweet",
)

  0%|          | 0/1 [00:00<?, ?it/s]

ValueError: Sum of sizes (5) must equal length of input array (6).

In [ ]:
tasks.run_helper.results(
    df2,
    discussion_id_col="fake_index",
    sdb_column="annotatorRace",
    value_col="racism",
    comment_key_col="tweet",
)

In [ ]:
tasks.run_helper.results(
    df2,
    discussion_id_col="fake_index",
    sdb_column="annotatorGender",
    value_col="racism",
    comment_key_col="tweet",
)